# Encoding Operations
This section contains;
* Rare Encoder
* Label Encoder
* One Hot Encoder

## Data Preparation

In [1]:
from pyspark.sql import SparkSession, functions as F
import findspark

findspark.init("C:\Program Files\Spark\spark-3.3.1-bin-hadoop3")

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
spark = SparkSession.builder \
    .appName("House Price Model") \
    .master("local[2]") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

In [4]:
path = "file:///Users/talha/OneDrive/Masaüstü/Talha Nebi Kumru/Data Science/Miuul/Machine Learning/House_Price_Prediction/datasets/outliers/non_outliers.parquet"
df = spark.read \
    .format("parquet") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load(path)

In [5]:
df.limit(5).toPandas()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60.0,RL,65.0,8450.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,8.0,Typ,0.0,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,WD,Normal,208500.0
1,20.0,RL,80.0,9600.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,TA,6.0,Typ,1.0,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,WD,Normal,181500.0
2,60.0,RL,68.0,11250.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,6.0,Typ,1.0,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,WD,Normal,223500.0
3,70.0,RL,60.0,9550.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,Gd,7.0,Typ,1.0,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,WD,Abnorml,140000.0
4,60.0,RL,84.0,14260.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,Gd,9.0,Typ,1.0,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,WD,Normal,250000.0


In [6]:
from _eda_utils import seperate_columns

In [7]:
numeric_cols, categoric_cols = seperate_columns(spark, df.dtypes)

2023-04-10 03:05:03,478 - logging_utils - DEBUG - seperate_columns function executing...
2023-04-10 03:05:03,481 - logging_utils - INFO - 
Numerical Columns: ['BsmtHalfBath', 'TotRmsAbvGrd', 'YrSold', '3SsnPorch', 'GarageArea', 'LotFrontage', 'YearBuilt', 'BsmtFullBath', 'PoolArea', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', 'FullBath', 'OverallCond', 'YearRemodAdd', 'WoodDeckSF', 'MoSold', '2ndFlrSF', 'MiscVal', 'OverallQual', 'TotalBsmtSF', 'BsmtFinSF1', 'OpenPorchSF', 'EnclosedPorch', 'LowQualFinSF', 'GarageYrBlt', 'LotArea', 'GrLivArea', 'ScreenPorch', 'MSSubClass', 'BedroomAbvGr', 'HalfBath', 'MasVnrArea', 'GarageCars', 'Fireplaces', 'SalePrice', 'KitchenAbvGr']
2023-04-10 03:05:03,482 - logging_utils - INFO - 
Categorical Columns: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Fo

In [8]:
from _sklearn_linear_regression import LinearRegressionSklearn

In [11]:
def get_score(df, target_col, spark=1):
    if spark: df = df.toPandas()
    reg = LinearRegressionSklearn(df, target_col='SalePrice')
    
    reg.initialize()
    
    reg.model()
    
    return reg.get_score()

In [12]:
get_score(df, 'SalePrice')

0.9096735738461703

## Rare Encoder

### Rare Analyser

In [13]:
from _encoders_utils import rare_analyser

In [14]:
rare_analyser(df, target='SalePrice', categoric_cols=categoric_cols)

2023-04-10 03:05:42,714 - logging_utils - DEBUG - rare_analyser function is executing...
+--------+-----+-----+-----------+
|MSZoning|Count|Ratio|Target Mean|
+--------+-----+-----+-----------+
|      RL| 1141| 0.79|  190518.73|
|      RM|  217| 0.15|  126645.48|
|      FV|   62| 0.04|  214888.13|
| C (all)|   10| 0.01|    74528.0|
|      RH|   16| 0.01|  131558.38|
+--------+-----+-----+-----------+

+------+-----+-----+-----------+
|Street|Count|Ratio|Target Mean|
+------+-----+-----+-----------+
|  Pave| 1441|  1.0|  180766.85|
|  Grvl|    5|  0.0|   110438.6|
+------+-----+-----+-----------+

+--------+-----+-----+-----------+
|LotShape|Count|Ratio|Target Mean|
+--------+-----+-----+-----------+
|     IR2|   41| 0.03|  239833.37|
|     Reg|  915| 0.63|  164256.78|
|     IR1|  481| 0.33|  206078.44|
|     IR3|    9| 0.01|  198373.89|
+--------+-----+-----+-----------+

+-----------+-----+-----+-----------+
|LandContour|Count|Ratio|Target Mean|
+-----------+-----+-----+-----------+
|

### Rare Encoding

In [16]:
from _encoders_utils import rare_encoder

In [17]:
temp_df = df

In [18]:
temp_df = rare_encoder(temp_df, columns=categoric_cols, rare_percent=0.01)

2023-04-10 03:06:28,039 - logging_utils - DEBUG - rare_analyser function is executing...
2023-04-10 03:06:28,107 - logging_utils - INFO - MSZoning column is encoding...
2023-04-10 03:06:29,102 - logging_utils - INFO - Street column is encoding...
2023-04-10 03:06:29,916 - logging_utils - INFO - LotShape column is encoding...
2023-04-10 03:06:30,698 - logging_utils - INFO - LandContour column is encoding...
2023-04-10 03:06:31,545 - logging_utils - INFO - Utilities column is encoding...
2023-04-10 03:06:32,342 - logging_utils - INFO - LotConfig column is encoding...
2023-04-10 03:06:33,106 - logging_utils - INFO - LandSlope column is encoding...
2023-04-10 03:06:33,864 - logging_utils - INFO - Neighborhood column is encoding...
2023-04-10 03:06:34,638 - logging_utils - INFO - Condition1 column is encoding...
2023-04-10 03:06:35,453 - logging_utils - INFO - Condition2 column is encoding...
2023-04-10 03:06:36,285 - logging_utils - INFO - BldgType column is encoding...
2023-04-10 03:06:37

#### Test Rare Encoded

In [19]:
get_score(temp_df, 'SalePrice')

0.9113329100844768

### Save

In [20]:
df = temp_df

In [21]:
df.toPandas().to_parquet('datasets/rare/rare_encoded.parquet', index=False)

### Conclusion

<div class="alert alert-block alert-info"> <b>Final:</b> After rare encoding, accuracy increased <b>0.9096735738461703<b> to <b>0.9113329100844768<b> </div>

## Label Encoding & One Hot Encoding

In [22]:
binary_cols = [col for col in categoric_cols if df.select(col).distinct().count() == 2]
one_hot_cols = [col for col in categoric_cols if col not in binary_cols]

In [23]:
len(binary_cols) + len(one_hot_cols) == len(categoric_cols)

True

### Label Encoding

In [24]:
from _encoders_utils import label_encoder

In [25]:
temp_df = df.toPandas()

In [26]:
temp_df = label_encoder(temp_df, binary_cols)

In [27]:
temp_df[binary_cols].head()

,Street,Utilities,Condition2,RoofMatl,Heating,CentralAir
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [28]:
temp_df.shape

(1446, 75)

In [30]:
df.count()

1446

In [32]:
df = temp_df

### One Hot Encoding

In [33]:
import pandas as pd

In [42]:
temp_df = df.copy()

In [43]:
temp_df = pd.get_dummies(temp_df, columns=one_hot_cols, drop_first=True)

In [44]:
temp_df.head()

,MSSubClass,LotFrontage,LotArea,Street,Utilities,Condition2,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofMatl,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,...,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_LwQ,BsmtFinType2_Rare,BsmtFinType2_Rec,BsmtFinType2_Unf,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Rare,HeatingQC_TA,Electrical_FuseF,Electrical_Rare,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Min2,Functional_Rare,Functional_Typ,GarageType_BuiltIn,GarageType_Detchd,GarageType_Rare,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Rare,GarageQual_TA,GarageCond_Rare,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_New,SaleType_Rare,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Rare
0,60.0,65.0,8450.0,0,0,0,7.0,5.0,2003.0,2003.0,0,196.0,706.0,0.0,150.0,856.0,0,1,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,8.0,0.0,2003.0,2.0,548.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0
1,20.0,80.0,9600.0,0,0,0,6.0,8.0,1976.0,1976.0,0,0.0,978.0,0.0,284.0,1262.0,0,1,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,6.0,1.0,1976.0,2.0,460.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0
2,60.0,68.0,11250.0,0,0,0,7.0,5.0,2001.0,2002.0,0,162.0,486.0,0.0,434.0,920.0,0,1,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,6.0,1.0,2001.0,2.0,608.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0
3,70.0,60.0,9550.0,0,0,0,7.0,5.0,1915.0,1970.0,0,0.0,216.0,0.0,540.0,756.0,0,1,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,7.0,1.0,1998.0,3.0,642.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0
4,60.0,84.0,14260.0,0,0,0,8.0,5.0,2000.0,2000.0,0,350.0,655.0,0.0,490.0,1145.0,0,1,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,9.0,1.0,2000.0,3.0,836.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0


In [45]:
df.shape

(1446, 75)

In [46]:
temp_df.shape

(1446, 171)

In [47]:
df = temp_df

## Final Test

In [48]:
get_score(df, 'SalePrice', 0)

0.9113344671996515

In [ ]:
df.to_parquet("datasets/encode/encoded.parquet", index=False)

## Conclusion

<div class="alert alert-block alert-info"> <b>Final:</b> Final encoding, accuracy increased <b>0.9096735738461703<b> to <b>0.9113329100844768<b> </div>

In [49]:
spark.stop()